In [30]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\Zodrick
[nltk_data]     John\AppData\Roaming\nltk_data...


True

In [32]:
def fetch_data_from_sql():
    conn_str = (
        "Driver={SQL Server};"
        "Server=ZODRICK-JOHN;"
        "Database=PortfolioProject_MarketingAnalytics;"
        "Trusted_Connection=yes;"
    )
    conn = pyodbc.connect(conn_str)
    query = "select ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText from customer_reviews"
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [34]:
customer_reviews_df = fetch_data_from_sql()
sia = SentimentIntensityAnalyzer()

C:\Users\Zodrick John\AppData\Local\Temp\ipykernel_25980\1890136154.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [35]:
# function to calculate sentiment scores using VADER
def calculate_sentiment(review):
    sentiment =sia.polarity_scores(review)
    return sentiment['compound'] # returning the compound score which is a normalized score between -1 and 1

In [36]:
# defined a function to categorize sentiment using both the sentiment score and the review rating
def categorize_sentiment(score, rating):
    if score > 0.05:
        if rating >= 4:
            return 'Positive'
        elif rating == 3:
            return 'Mixed Positive'
        else :
            return 'Mixed Negative'
    elif score < -0.05:
        if rating <= 2:
            return "Negative"
        elif rating == 3:
            return "Mixed Negative"
        else:
            return "Mixed Positive"
    else:
        if rating >= 4:
            return "Postive"
        elif rating <=2:
            return "Negative"
        else:
            return "Neutral"

In [37]:
# defined a function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'
    else:
        return '-1.0 to -0.5'

In [38]:
# apply sentiment analysis to calculate sentiment scores for each review
customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)
# apply sentiment categorization using both text and rating
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'],row['Rating']), axis=1)

In [45]:
print(customer_reviews_df.head())

   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                                 ReviewText  SentimentScore SentimentCategory  
0   Average  experience,  nothing  special.         -0.3089    Mixed Negative  
1            The  quality  is    top-notch.          0.0000           Postive  
2   Five  stars  for  the  quick  delivery.          0.0000           Postive  
3  Good  quality,  but  could  be  cheaper.          0.2382    Mixed Positive  
4   Average  experience,  nothing  special.         -0.3089    Mixed Negative  


In [47]:
customer_reviews_df.to_csv('fact_customer_reviews_with_sentiment.csv', index=False)